### Training Vlads model by first stacking the depths from the Sota models

In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from configs import extended_unet_fusion_config as config
from models.unet_mit import Unet
from datasets.combination_depth_dataset import CombDepthDataset
from models.fusion_models import TransformerFusionModel
from models.combination_model import CombinedModel
from utils.train_utils import train_model
import utils.train_utils as tu

/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    depths_dir=config.depth_maps_path,
    list_file=os.path.join(config.dataset_path, 'train_list.txt'),
    transform=config.padded_transform,
    target_transform=config.target_transform,
    has_gt=True,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)
    
    # Create test dataset without ground truth
test_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    depths_dir=os.path.join(config.depth_maps_path, 'test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.padded_transform,
    has_gt=False,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation


# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True,
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [3]:
model = config.model()




Num channels: 4


In [4]:
fusion_model = TransformerFusionModel(input_channels=len(test_dataset.depth_model_names)).to(config.device)
model = CombinedModel(fusion_model=fusion_model, unet_model=model, use_uncertainty=False).to(config.device)

In [5]:
optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Using device: cuda:3


In [6]:
#exp_name = "extended_unet_sota_depths"
#model.load_state_dict(torch.load(f'{os.path.join(config.dataset_path, exp_name)}/best_model_43.pt'))

In [ ]:
# Wandb logging
exp_name = "extended_unet_sota_depths_fusion_transformer_pretrain"

print("Starting training...")
with wandb.init(project="MonocularDepthEstimation",  #CIL
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, 35, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name),
                       is_extended_model=True, use_uncertainty_map=False)

Starting training...


wandb: Currently logged in as: noloo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/35


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:28<00:00,  2.55it/s]


Train Loss: 0.4397, Validation Loss: 0.6493
New best model saved at epoch 1 with validation loss: 0.6493
Epoch 2/35


...

Training:   3%|████▏                                                                                                                                                      | 68/2546 [00:37<21:54,  1.88it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:27<00:00,  2.58it/s]


Train Loss: 0.1058, Validation Loss: 0.1116
New best model saved at epoch 27 with validation loss: 0.1116
Epoch 28/35


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:26<00:00,  2.60it/s]


Train Loss: 0.1045, Validation Loss: 0.1107
New best model saved at epoch 28 with validation loss: 0.1107
Epoch 29/35


Training:   6%|█████████▊                                                                                                                                                | 163/2546 [01:27<21:02,  1.89it/s]

In [8]:
tu.generate_test_predictions(model, test_loader, config.device,
                          exp_path=os.path.join(config.dataset_path, exp_name),
                            is_exntended_model=True)

Generating Test Predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:16<00:00,  2.53it/s]


In [ ]:

importlib.reload(tu)
tu.visualize_test_predictions(model, test_loader, config.device,
                              exp_path=os.path.join(config.dataset_path, exp_name), is_exntended_model=True)